In [128]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
import string
import collections
import random
import time
import nltk
from nltk import word_tokenize
from textaugment import Wordnet

In [129]:
local_api_url = 'http://localhost:8080/resource/skill'

In [ ]:
# Not needed for now, just saved here in case it's needed for later -- also Silenced

# search for all jobs that have 'manager' in their title  --- IGNORE this cell for now
# WEB API URL (in case you don't want to use the local API): url_title = "https://ec.europa.eu/esco/api/search"
# also: url = "https://ec.europa.eu/esco/api/resource/skill"

params = {
    'language': 'en',
    'type': 'occupation',
    'text': 'pig'
         }

api_title = requests.get(url=url_title, params=params).json()

In [ ]:
# create an augmented dataframe by appending rows for each alternative label of each job title
# and uses the Text Augmentation model to create synonym descriptions for each alternative label
# DON'T RUN THIS, IT'S BEEN RUN ALREADY -- it can only to be run once because it takes a while :)
# (also, this cell has been set to Silence)

# get word thesaurus

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# initialize augment model

t = Wordnet(v=False ,n=True, p=0.75)

# run the API sweeping loop

df_occupations_aug = df_occupations

for index, row in df_occupations.iterrows():

    if pd.notna(df_occupations.iloc[index,4]):
        
        list_of_altLabels_per_row = row[4].split('\n')        
        
        for altLabel in list_of_altLabels_per_row:
            
            df_of_separate_row = pd.DataFrame([row])
            df_occupations_aug = pd.concat([df_occupations_aug, df_of_separate_row])
            print(f"Job Title {index}: Appended {row[3]}'s alternative label: {altLabel}")
            df_occupations_aug.iloc[-1,3] = altLabel
            df_occupations_aug.iloc[-1,12] = t.augment(row[12])

# write augmented occupations dataframe to CSV file
df_occupations_aug.to_csv('../data/ESCO/occupations_augmented.csv')

In [130]:
df_occupations_aug = pd.read_csv('../data/ESCO/occupations_augmented.csv')

In [131]:
df_occupations_aug['skills'] = 0

In [132]:
### create a separate skill column on the augmented dataframe for the skills got from the local API

for index, row in tqdm(df_occupations_aug.iterrows()):
    
    params = {'uri': df_occupations_aug.iloc[index,2], 'language': 'en'}
    api_skills = requests.get(url=local_api_url, params=params).json()
    api_skills_dict = api_skills.get('_links').get('hasEssentialSkill')
    
    try:
        skills_list = [skill.get('title') for skill in api_skills_dict]
    except:
        pass
        
    df_occupations_aug.iloc[index,15] = ', '.join(skills_list) 

35824it [56:35, 10.55it/s]


In [149]:
df_occupations_aug.to_csv('../data/ESCO/occupations_augmented_with_skills.csv')
